In [1]:
import sqlite3

# Connexion à la base de données (ou création de celle-ci)
conn = sqlite3.connect('Gestion_eau.db')
cursor = conn.cursor()

In [13]:
import sqlite3

# Connexion à la base de données SQLite
conn = sqlite3.connect('Gestion_eau.db')
cursor = conn.cursor()

# Liste des tables à supprimer
tables = [
    "info_personne",
    "Abonnement",
    "Qte_Consommation",
    "Pay_Consommation",
    "Produit_Acheter",
    "Produit",
    "Produit_Utiliser",
    "ONEP_Payment",
    "ONEP_Credit",
    "Maintenance",
    "Maintenance_Details",
    "Blocks",
    "Achat"
]

# Générer et exécuter les commandes DROP TABLE IF EXISTS
for table in tables:
    cursor.execute(f"DROP TABLE IF EXISTS {table}")
    print(f"Table '{table}' supprimée.")

# Valider les modifications et fermer la connexion
conn.commit()
conn.close()


Table 'info_personne' supprimée.
Table 'Abonnement' supprimée.
Table 'Qte_Consommation' supprimée.
Table 'Pay_Consommation' supprimée.
Table 'Produit_Acheter' supprimée.
Table 'Produit' supprimée.
Table 'Produit_Utiliser' supprimée.
Table 'ONEP_Payment' supprimée.
Table 'ONEP_Credit' supprimée.
Table 'Maintenance' supprimée.
Table 'Maintenance_Details' supprimée.
Table 'Blocks' supprimée.
Table 'Achat' supprimée.


In [58]:
cursor.execute('''
CREATE VIEW IF NOT EXISTS Mouvements_Caisse AS
SELECT N_contrat AS ID, Date_payement AS Date_Mouvement, 'Adhision' AS Motif, Mnt_paye AS Montant
FROM Abonnement
UNION ALL
SELECT N_contrat AS ID, Date_payement AS Date_Mouvement, 'Consommation' AS Motif, Mnt_paye AS Montant
FROM Pay_Consommation
UNION ALL
SELECT ID_Payment AS ID, Date_Reglement AS Date_Mouvement, 'ONEP' AS Motif, Mnt_paye AS Montant
FROM ONEP_Payment
UNION ALL
SELECT ID_Maintenance AS ID, Date_operation AS Date_Mouvement, 'Maintenance' AS Motif, Mnt_ouvrier AS Montant 
FROM Maintenance
UNION ALL
SELECT ID_Achat AS ID, Date_Achat AS Date_Mouvement, 'Charge Materiel' AS Motif, -Montant_total AS Montant
FROM Produit_Acheter
''')


In [55]:
cursor.execute("DROP VIEW IF EXISTS Mouvements_Caisse")


In [2]:
# Étape 1 : Créer une table temporaire sans clé primaire
cursor.execute('''
CREATE TABLE IF NOT EXISTS Blocks_temp (
    Block_ID INTEGER,
    Index_m3 REAL,
    Date_consome DATE
);
''')

# Étape 2 : Copier les données de l'ancienne table vers la nouvelle table
cursor.execute('''
INSERT INTO Blocks_temp (Block_ID, Index_m3, Date_consome)
SELECT Block_ID, Index_m3, Date_consome FROM Blocks;
''')

# Étape 3 : Supprimer l'ancienne table
cursor.execute('DROP TABLE Blocks;')

# Étape 4 : Renommer la table temporaire
cursor.execute('ALTER TABLE Blocks_temp RENAME TO Blocks;')

# Sauvegarder les modifications
conn.commit()
conn.close()

print("La clé primaire a été supprimée avec succès.")



La clé primaire a été supprimée avec succès.


In [ ]:

cursor.execute('''DELETE FROM ONEP_Payment
WHERE Recu = 4444;''')


In [2]:
cursor.execute('''
INSERT INTO Produit (Nom_Produit) VALUES
    ('Compteur'),
    ('Pompe'),
    ('Filtre'),
    ('Réservoir'),
    ('Tuyau irrigation'),
    ('Capteur de niveau'),
    ('Vanne à bille'),
    ('Distributeur'),
    ('Adoucisseur'),
    ('Kit de traitement'),
    ('Divers');
''')


# Table 

In [14]:
import sqlite3

# Connexion à la base de données (ou création de celle-ci)
conn = sqlite3.connect('Gestion_eau.db')
cursor = conn.cursor()

# Création de la table info_personne
cursor.execute('''
CREATE TABLE IF NOT EXISTS info_personne (
    N_contrat TEXT PRIMARY KEY,
    Cin TEXT,
    N_conpteur_B TEXT,
    N_conpteur_P TEXT,
    Nom TEXT NOT NULL,
    Mnt_due REAL,
    Date_Adhesion DATE,
    Adresse TEXT
)
''')

# Création de la table Abonnement
cursor.execute('''
CREATE TABLE IF NOT EXISTS Abonnement (
    N_contrat TEXT,
    N_recue TEXT,
    Mnt_paye REAL,
    Date_payement DATE,
    FOREIGN KEY (N_contrat) REFERENCES info_personne(N_contrat)
)
''')

# Création de la table Qte_Consommation
cursor.execute('''
CREATE TABLE IF NOT EXISTS Qte_Consommation (
    N_contrat TEXT,
    Date_consome DATE,
    Quantite REAL,
    FOREIGN KEY (N_contrat) REFERENCES info_personne(N_contrat)
)
''')

# Création de la table Pay_Consommation
cursor.execute('''
CREATE TABLE IF NOT EXISTS Pay_Consommation (
    N_contrat TEXT,
    N_recue TEXT,
    Mnt_paye REAL,
    Date_paye Date,
    Date_payement DATE,
    FOREIGN KEY (N_contrat) REFERENCES info_personne(N_contrat))
''')

"""# Table Mouvement_Stock 
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Mouvement_Stock (
    ID_Mouvement INTEGER PRIMARY KEY AUTOINCREMENT,
    Nom_Produit TEXT REFERENCES Produit(Nom_Produit),
    Date_Mouvement DATE,
    Type_Mouvement TEXT,  -- Par exemple : "entrée", "sortie", "retour"
    Quantite REAL,  -- Quantité ajoutée ou retirée
    Description TEXT  -- Détails du mouvement (par exemple : "Sortie pour installation")
)
''')"""

# Table Achat 
cursor.execute('''
CREATE TABLE IF NOT EXISTS Produit_Acheter (
    ID_Achat INTEGER PRIMARY KEY AUTOINCREMENT,
    Nom_Produit TEXT REFERENCES Produit(Nom_Produit),
    Date_Achat DATE,
    Quantite_achetee REAL,
    Prix_unitaire REAL,
    Montant_total REAL AS (Quantite_achetee * Prix_unitaire),
    N_recu TEXT,
    Fournisseur TEXT
)
''')

# Table Produit
cursor.execute('''
CREATE TABLE IF NOT EXISTS Produit (
    Nom_Produit TEXT PRIMARY KEY NOT NULL
)
''')

# Table Produit_Utiliser
cursor.execute('''
CREATE TABLE IF NOT EXISTS Produit_Utiliser (
    ID_utils INTEGER PRIMARY KEY AUTOINCREMENT,
    Nom_Produit TEXT REFERENCES Produit(Nom_Produit),
    Date_utils DATE,
    Quantite_utuli REAL,
    Description TEXT
)
''')

#Table ONEP_Payement
cursor.execute('''CREATE TABLE IF NOT EXISTS ONEP_Payment (
    ID_Payment INTEGER PRIMARY KEY AUTOINCREMENT, 
    Mois_consome TEXT NOT NULL,            
    Mnt_paye REAL NOT NULL,                
    Recu TEXT NOT NULL,                    
    Date_Reglement DATE NOT NULL                  
)''')

# Table ONEP_Charge
cursor.execute('''CREATE TABLE IF NOT EXISTS ONEP_Credit (
    ID_Charge INTEGER PRIMARY KEY AUTOINCREMENT,  
    Mois_consome TEXT NOT NULL,  
    Quantite_Consome REAL,
    Mnt_a_paye REAL NOT NULL,                
    Facture TEXT                             
) ''')

# Table Maintenance
cursor.execute(''' CREATE TABLE IF NOT EXISTS Maintenance (
    ID_Maintenance INTEGER PRIMARY KEY AUTOINCREMENT,
    Date_operation DATE NOT NULL,
    Description TEXT,
    Nom_ouvrier TEXT,
    Mnt_ouvrier REAL
)
''')
               
# Table Maintenance en detail
cursor.execute(''' CREATE TABLE IF NOT EXISTS Maintenance_Details (
    ID_Detail INTEGER PRIMARY KEY AUTOINCREMENT,
    ID_Maintenance INTEGER REFERENCES Maintenance(ID_Maintenance) ON DELETE CASCADE,
    Nom_produit TEXT REFERENCES Produit(Nom_Produit),
    Quantite_produit REAL NOT NULL,
    FOREIGN KEY (ID_Maintenance) REFERENCES Maintenance(ID_Maintenance)
)
''')

cursor.execute('''CREATE TABLE IF NOT EXISTS Blocks (
    Block_ID INTEGER ,
    Index_m3 REAL NOT NULL,
    Date_consome Date
);
       ''')

# Inserion 

In [15]:
import sqlite3

# Connexion à la base de données
conn = sqlite3.connect('Gestion_eau.db')
cursor = conn.cursor()

# Données à insérer
data_info_personne = [
    ('01', 'Bk234', '1', 'CP001', 'John Doe', 1500.50, '2024-01-01', 'hamou'),
    ('02', 'BK123', '2', 'CP002', 'Jane Smith', 2000.00, '2024-02-01', 'baala'),
    ('03', 'BK45', '3', 'CP003', 'Ahmed Ali', 1750.75, '2024-03-10', '789 Boulevard Zerktouni, Fès'),
    ('04', 'CIN004', '2', 'CP004', 'Sarah Karim', 1850.00, '2024-03-15', '321 Rue Ibn Battuta, Marrakech'),
    ('05', 'CIN005', '3', 'CP005', 'Youssef Haddad', 1450.25, '2024-04-01', '654 Avenue Hassan II, Tanger'),
    ('06', 'CIN006', '4', 'CP006', 'Fatima Zahra', 2100.00, '2024-04-20', '876 Rue Oqba Ibn Nafia, Agadir'),
    ('07', 'CIN007', '4', 'CP007', 'Mohamed Chraibi', 1950.30, '2024-05-01', '543 Boulevard Moulay Youssef, Oujda'),
    ('08', 'CIN008', '4', 'CP008', 'Hajar El Alaoui', 1800.00, '2024-05-10', '234 Rue Al Mounir, Meknès'),
    ('09', 'CIN009', '1', 'CP009', 'Rachid Benomar', 1650.60, '2024-06-01', '123 Rue Dakhla, Tétouan'),
    ('10', 'CIN010', '3', 'CP010', 'Samira Lahlou', 1750.80, '2024-06-15', '456 Avenue Abdelmoumen, Laâyoune')
]

# Insertion des données
cursor.executemany('''
INSERT INTO info_personne (N_contrat, Cin, N_conpteur_B, N_conpteur_P, Nom, Mnt_due, Date_Adhesion, Adresse)
VALUES (?, ?, ?, ?, ?, ?, ?, ?)
''', data_info_personne)

# Validation des modifications et fermeture de la connexion
conn.commit()
conn.close()

print("Données insérées avec succès.")


Données insérées avec succès.


In [16]:
import sqlite3

# Connexion à la base de données
conn = sqlite3.connect('Gestion_eau.db')
cursor = conn.cursor()

# Données à insérer dans la table Abonnement
data_abonnement = [
    ('01', '1', 500.00, '2024-01-10'),
    ('01', '2', 500.00, '2024-02-10'),
    ('01', '3', 500.00, '2024-03-10'),
    ('01', '4', 0.50, '2024-04-01'),
    ('02', '5', 1000.00, '2024-02-15'),
    ('02', '6', 1000.00, '2024-03-15'),
    ('03', '7', 500.00, '2024-03-15'),
    ('03', '8', 500.00, '2024-04-05'),
    ('04', '9', 800.00, '2024-03-20'),
    ('04', '10', 800.00, '2024-04-05'),
    ('05', '11', 800.00, '2024-04-10'),
    ('06', '12', 500.00, '2024-04-25'),
    ('06', '13', 500.00, '2024-05-05'),
    ('07', '14', 1000.00, '2024-05-15'),
    ('08', '15', 1800.00, '2024-05-25'),
    ('09', '16', 1650.60, '2024-06-10'),
    ('10', '17', 600.00, '2024-06-20'),
    ('10', '18', 1150.80, '2024-07-01')
]

# Insertion des données dans la table Abonnement
cursor.executemany('''
INSERT INTO Abonnement (N_contrat, N_recue, Mnt_paye, Date_payement)
VALUES (?, ?, ?, ?)
''', data_abonnement)

# Validation des modifications et fermeture de la connexion
conn.commit()
conn.close()

print("Données insérées avec succès dans la table Abonnement.")


Données insérées avec succès dans la table Abonnement.


In [8]:
import sqlite3

# Connexion à la base de données
conn = sqlite3.connect('Gestion_eau.db')
cursor = conn.cursor()

# Données de consommation à insérer
data_consommation = [
    ('01', 2, '2024-06-01'),
    ('01', 4, '2024-07-01'),
    ('01', 5, '2024-08-01'),
    ('01', 6, '2024-09-01'),
    ('01', 7, '2024-10-01'),
    ('01', 10, '2024-11-01'),
    ('02', 3, '2024-06-01'),
    ('02', 9, '2024-07-01'),
    ('02', 14, '2024-08-01'),
    ('02', 18, '2024-09-01'),
    ('02', 19, '2024-10-01'),
    ('02', 22, '2024-11-01'),
    ('03', 1, '2024-06-01'),
    ('03', 7, '2024-07-01'),
    ('03', 10, '2024-08-01'),
    ('03', 11, '2024-09-01'),
    ('03', 15, '2024-10-01'),
    ('03', 20, '2024-11-01'),
    ('04', 10, '2024-06-01'),
    ('04', 13, '2024-07-01'),
    ('04', 19, '2024-08-01'),
    ('04', 22, '2024-09-01'),
    ('04', 26, '2024-10-01'),
    ('04', 28, '2024-11-01'),
    ('05', 1, '2024-06-01'),
    ('05', 7, '2024-07-01'),
    ('05', 9, '2024-08-01'),
    ('05', 17, '2024-09-01'),
    ('05', 20, '2024-10-01'),
    ('05', 28, '2024-11-01')
]

# Insertion des données dans la table Consommation
cursor.executemany('''
INSERT INTO Qte_Consommation (N_contrat, Quantite, Date_consome)
VALUES (?, ?, ?)
''', data_consommation)

# Validation des modifications et fermeture de la connexion
conn.commit()
conn.close()


In [11]:
import sqlite3

# Connexion à la base de données
conn = sqlite3.connect('Gestion_eau.db')
cursor = conn.cursor()

# Données à insérer
data_pay_consommation = [
    ('01', '1', 14.00, '2024-06-15', '2024-06-16'),
    ('01', '6', 14.00, '2024-07-01', '2024-07-02'),
    ('01', '5', 7.00, '2024-08-15', '2024-08-16'),
    ('01', '6', 7.00, '2024-09-01', '2024-09-02'),
    ('01', '2', 7.00, '2024-10-15', '2024-10-16'),

    ('02', '2', 21.00, '2024-06-15', '2024-06-16'),
    ('02', '7', 42.00, '2024-07-01', '2024-07-02'),
    ('02', '11', 35.00, '2024-08-15', '2024-08-16'),
    ('02', '15', 28.00, '2024-09-01', '2024-09-02'),
    
    ('03', '9', 7.00, '2024-06-15', '2024-06-16'),
    ('03', '16', 42.00, '2024-07-01', '2024-07-02'),
    ('03', '15', 21.00, '2024-08-15', '2024-08-16'),
    ('03', '16', 7.00, '2024-09-01', '2024-09-02'),
    ('03', '12', 28.00, '2024-10-15', '2024-10-16'),
    ('03', '10', 35.00, '2024-11-15', '2024-11-16'),

    ('04', '3', 21.00, '2024-09-01', '2024-10-02'),
    ('04', '7', 70.00, '2024-06-15', '2024-08-16'),
    ('04', '12', 14.00, '2024-11-01', '2024-11-02'),

    ('05', '12', 7.00, '2024-06-01', '2024-06-02'),
    
]

# Insertion des données
cursor.executemany('''
INSERT INTO Pay_Consommation (N_contrat, N_recue, Mnt_paye, Date_paye, Date_payement)
VALUES (?, ?, ?, ?, ?)
''', data_pay_consommation)

# Validation des changements
conn.commit()

# Fermeture de la connexion
conn.close()

print("Les données ont été insérées dans la table Pay_Consommation avec succès.")


Les données ont été insérées dans la table Pay_Consommation avec succès.


# XXX

In [5]:
cursor.execute("DROP VIEW IF EXISTS Mouvements_Caisse")

# Valider et fermer la connexion
conn.commit()
conn.close()

In [ ]:
import sqlite3

# Connexion à la base de données (remplacez par votre connexion de base de données)
conn = sqlite3.connect('Gestion_eau.db')
cursor = conn.cursor()

# Exécuter la requête SQL
query = '''
SELECT 
    q.N_contrat,
    strftime('%Y-%m', q.Date_consome) AS Mois_Consome,
    q.Date_consome,
    q.Quantite AS Index_m3,
    COALESCE(q.Quantite - LAG(q.Quantite) OVER (PARTITION BY q.N_contrat ORDER BY q.Date_consome), q.Quantite) AS Qte_Consomme_m3,
    (COALESCE(q.Quantite - LAG(q.Quantite) OVER (PARTITION BY q.N_contrat ORDER BY q.Date_consome), q.Quantite) * 7) AS Montant_dh
FROM 
    Qte_Consommation q;

'''

cursor.execute(query)

# Récupérer les résultats
results = cursor.fetchall()

# Afficher les résultats
for row in results:
    print(row)

# Fermer la connexion
conn.close()


('000', '2024-12', '2024-12-01', 90.0, 90.0)
('100', '2024-11', '2024-11-01', 0.0, 0.0)
('100', '2024-11', '2024-11-01', 0.0, 0.0)
('11', '2024-08', '2024-08-01', 60.0, 60.0)
('111', '2024-10', '2024-10-01', 70.0, 70.0)
('111', '2024-11', '2024-11-01', 77.0, 7.0)
('111', '2024-11', '2024-11-13', 129.0, 52.0)
('111', '2024-12', '2024-12-01', 90.0, -39.0)
('123', '2024-11', '2024-11-01', 301.0, 301.0)
('123', '2024-12', '2024-12-01', 12345.0, 12044.0)
('124', '2024-11', '2024-11-01', 56.0, 56.0)
('222', '2024-11', '2024-11-07', 145.0, 145.0)
('222', '2024-12', '2024-12-03', 160.0, 15.0)
('222', '2025-01', '2025-01-02', 179.0, 19.0)
('333', '2024-10', '2024-10-01', 50.0, 50.0)
('333', '2024-11', '2024-11-02', 56.0, 6.0)
('55', '2024-09', '2024-09-01', 5.0, 5.0)
('55', '2024-10', '2024-10-01', 10.0, 5.0)
('55', '2024-11', '2024-11-01', 15.0, 5.0)
('78', '2024-08', '2024-08-01', 12.0, 12.0)
('78', '2024-09', '2024-09-02', 18.0, 6.0)
('78', '2024-10', '2024-10-03', 25.0, 7.0)
('78', '2024-11

In [13]:
import sqlite3

# Connexion à la base de données
conn = sqlite3.connect('Gestion_eau.db')

# Création d'un curseur
cursor = conn.cursor()

# Exécution de la requête pour lister les tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

# Récupération et affichage des noms de tables
tables = cursor.fetchall()
print("Tables dans la base de données Gestion_eau :")
for table in tables:
    print(table[0])

# Fermeture de la connexion
conn.close()


Tables dans la base de données Gestion_eau :
info_personne
Abonnement
Qte_Consommation
Pay_Consommation


In [3]:
# Enregistrer les modifications et fermer la connexion
conn.commit()
conn.close()